# Fusion retrieval in document search

This notebook demonstrates the implementation of fusion retrieval in a document search system. Traditional retrieval systems typically use either semantic search (vector-based) or keyword-based search (BM25). However, each approach has its limitations:
- Vector search (semantic search) captures the meanings and concepts of documents, but can be less precise when exact matches are required.
- BM25 search (keyword-based) is great for finding exact keyword matches, but it doesn't account for semantic meaning or context.

The goal is to combine vector-based similarity search with keyword-based BM25 retrieval to improve the overall accuracy and relevance of document retrieval. By leveraging both semantic understanding (from the vector search) and exact keyword matching (from BM25), the system provides a more robust and comprehensive retrieval process.

In [1]:
import os
from dotenv import load_dotenv
from langchain.docstore.document import Document
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

from typing import List
from rank_bm25 import BM25Okapi
import numpy as np

# Load environment variables from a .env file
load_dotenv()

# Access the API key
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

### Loading the PDF
We will use `PyPDFLoader` to extract text from the PDF. It reads the PDF page by page and stores the extracted text in a list of document objects, where each document contains the content of a single page.

In [2]:
# Path to the PDF document
path = "Understanding_Climate_Change.pdf"

# Load PDF documents
loader = PyPDFLoader(path)
documents = loader.load()

### Preprocessing

#### Splitting documents into chunks
We use the `RecursiveCharacterTextSplitter` to split the document into smaller chunks. This is ideal when working with large documents to make them manageable for embedding generation and for easier retrieval.

In [3]:
# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, length_function=len)
texts = text_splitter.split_documents(documents)

we are splitting the document into chunks of size 1000 characters with 200 characters of overlap between chunks.
- `chunk_size` makes it more manageable for indexing and retrieval.
- chunk_overlap` ensures that the context is preserved when the text is split. This helps maintain the flow of information between chunks.
- `length_function` tells the splitter to calculate the length of the chunks based on the number of characters, ensuring that the chunks are exactly the specified size.

#### Replace tabs with spaces
In many cases, PDFs may contain tab characters (`\t`) that were used for indentation but aren't necessary for the final processed text. We will replace these with spaces.

In [4]:
# Replace tab characters ('\t') with spaces in the document chunks
for doc in texts:
    doc.page_content = doc.page_content.replace('\t', ' ')  # Replace tabs with spaces

Now, the document is ready for both semantic and keyword-based search.

### Creating the vector store
Once the text is cleaned and processed, we can create embeddings for each of the chunks using OpenAI API. These embeddings represent the meaning of the text in a high-dimensional vector space.

In [5]:
# Initializes the OpenAI embeddings model
embeddings = OpenAIEmbeddings()

We will also use FAISS to efficiently store and index the embeddings, which allows us to perform similarity search and query efficiently.

In [6]:
# Create vector store using FAISS
vector_store = FAISS.from_documents(texts, embeddings)

Here, we create a FAISS vector store by:
- Generating embeddings for each chunk of text.
- Storing the embeddings in FAISS, which allows fast similarity search later.

This function automatically creates a flat (brute-force) index by default.

### Creating the BM25 index for keyword-based retrieval
Next, we will create a BM25 index using the BM25Okapi class from the `rank_bm25` module. BM25 is a algorithm for ranking documents based on keyword matching. The main idea behind BM25 is to score documents based on how well they match the query words, adjusting for factors like term frequency and document length.

Before we can create the BM25 index, we need to tokenize the text of each document by splitting it into words. This will allow BM25 to process the documents efficiently.

In [7]:
# Tokenize each document by splitting on whitespace
tokenized_docs = [doc.page_content.split() for doc in texts]

# Create BM25 index from the tokenized documents
bm25 = BM25Okapi(tokenized_docs)

When we create a BM25 index, it pre-processes the documents in our collection (corpus) and creates a structure that allows fast retrieval based on the frequency of terms in the documents.

Once the BM25 index is created, the BM25 object can be used to score documents based on a query. Specifically, you can use the `get_scores()` method to calculate the BM25 scores for all documents in the index given a query.

### Fusion retrieval: Combining vector and BM25 retrieval
The key part of this system is the fusion retrieval method, which combines the results from the vector-based search and the BM25-based search. This method performs both searches and ranks documents based on a weighted combination of the scores from each method.

In [8]:
# Define query
query = "What are the impacts of climate change on the environment?"

#### Vector-based search
We begin by performing a vector-based search using the FAISS vector store.

- First, we retrieve all documents from the vector store.
- Then, we proceed to perform a vector-based search for the query.This search finds the documents that are semantically most similar to the given query.

In [11]:
# 1. Get all documents from the vectorstore
all_docs = vector_store.similarity_search("", k=vector_store.index.ntotal)

# Perform vector-based search
vector_results = vector_store.similarity_search_with_score(query, k=len(all_docs))

1. Here, we are calling the `similarity_search()` method on the vectorstore with an empty query (`""`), effectively retrieving all the documents in the vector store. We specify `k=vectorstore.index.ntotal` to fetch all the documents stored in the FAISS index. This gives us the complete set of documents available for retrieval.
2. Here, we use the `similarity_search_with_score()` method to search for the most relevant documents based on their vector embeddings and return the top documents along with their similarity scores. These scores represent how semantically similar each document is to the query. The argument `k=len(all_docs)` is used to retrieve the same number of documents as there are in the entire vector store.
  - In the context of fusion retrieval, we need to consider every document in the vector store to make sure we can later combine the results from both vector-based search (using FAISS) and BM25 search. By retrieving all documents, we ensure that both search methods (vector-based and BM25) can process the same set of documents, allowing for proper fusion of their results.

#### BM25-based search
Alongside the vector search, we also perform a BM25-based search using the previously created BM25 index. The BM25 algorithm works by matching the query terms with the terms in the documents, using statistical measures like term frequency (TF) and inverse document frequency (IDF) to rank the documents.

In [12]:
# Perform BM25 search
bm25_scores = bm25.get_scores(query.split())

The query is first split into individual terms using the `split()` method. Then, the `get_scores()` method from the BM25 index is called to compute the BM25 scores for each document. These scores are based on how well the query terms match the words in each document, adjusted for factors like term frequency and document length.

#### Normalizing the scores
To combine the results from both the vector-based search and the BM25-based search, we first normalize the scores from both methods. This ensures that the scores are on the same scale, making them comparable.

In [13]:
epsilon = 1e-8

# Normalize vector scores
vector_scores = np.array([score for _, score in vector_results])
vector_scores = 1 - (vector_scores - np.min(vector_scores)) / (np.max(vector_scores) - np.min(vector_scores) + epsilon)

# Normalize BM25 scores
bm25_scores = (bm25_scores - np.min(bm25_scores)) / (np.max(bm25_scores) - np.min(bm25_scores) + epsilon)

This is done by subtracting the minimum score and dividing by the range (max - min). Normalization ensures that the scores are comparable even though they might come from different scoring systems. `epsilon` is used to prevent a division by zero error in case the maximum and minimum values of the vector scores are identical (i.e., all the scores are the same).

#### Combining the scores
Now, we combine the normalized scores from both methods using a weighted sum. The weight alpha controls the balance between the two methods. A value of `alpha=0.5` means both methods contribute equally to the final score.

In [14]:
# Combine vector and BM25 scores using a weighted sum (alpha controls the balance)
alpha = 0.5  # 0.5 means equal weight for both methods
combined_scores = alpha * vector_scores + (1 - alpha) * bm25_scores

The `alpha` parameter controls the weight given to the vector-based search, and `(1 - alpha)` controls the weight for the BM25 search. By adjusting alpha, we can fine-tune how much influence each method has on the final ranking.

#### Ranking and retrieving the top documents
Finally, we rank the documents based on the combined scores and retrieve the top `k` documents.

In [15]:
# Rank documents based on combined scores
sorted_indices = np.argsort(combined_scores)[::-1]

# Retrieve the top k documents based on the combined scores
top_docs = [all_docs[i] for i in sorted_indices[:5]]

- The `np.argsort()` function sorts the combined scores in descending order. The result is the indices of the documents, sorted by their combined relevance scores.
- Using the sorted indices, we select the top 5 documents from the original set of documents (`all_docs`) that were retrieved earlier. These are the documents that are deemed most relevant to the query based on the combined scores.

#### Displaying the results
Now that we have the top `k` documents based on the fusion of vector and BM25 retrieval, we can display the content of these documents.

In [16]:
# Show the content of the top documents
docs_content = [doc.page_content for doc in top_docs]
docs_content

['a long time. These projects can help sequester carbon and provide new habitats for wildlife. \nStrategic planning and ecological considerations are essential for maximizing benefits. \nClimate Policy \nEffective climate policy is essential for driving large-scale change. International agreements, \nsuch as the Paris Agreement, aim to limit global warming to well below 2 degrees Celsius \nabove pre-industrial levels. National and local policies also play a critical role in \nimplementing mitigation and adaptation strategies. \nInternational Agreements \nInternational climate agreements, such as the Kyoto Protocol and the Paris Agreement, set \ntargets and frameworks for reducing greenhouse gas emissions globally. Cooperation and \ncommitment from all countries are necessary for achieving climate goals. \nNational Policies',
 'arid and semi-arid regions. Droughts can lead to food and water shortages and exacerbate \nconflicts. \nFlooding \nHeavy rainfall events are becoming more common